# ETL

In [23]:
#Importamos los paquete necesarios:
import os
import pandas as pd
import glob ##New package learned
import shutil ##New package learned   
from sqlalchemy import create_engine

# Extract:


### Extracción dependiendo del tipo de formato

In [ ]:
def extractANDclean_CSV(file_csv):
    '''Transformamos un CSV en un datafram y lo renombramos como documento procesado
    
    IN: archivo CSV
    OUT: DataFrame
    '''
    return pd.read_csv(file_csv)

def extractANDclean_JSON(file_json):
    '''Transformamos un JSON en un datafram y lo renombramos como documento procesado

    IN: archivo JSON
    OUT: DataFrame
    '''
    return pd.read_json(file_json, lines=True) ###REMARK

def extractANDclean_TXT(file_txt):
    '''Transformamos un TXT en un datafram y lo renombramos como documento procesado

    IN: archivo TXT
    OUT: DataFrame
    '''
    return pd.read_csv(file_txt,sep='|') ###REMARK

def extractANDclean_XLSX(file_xlsx):
    '''Transformamos un XLSX en un datafram y lo renombramos como documento procesado

    IN: archivo XLSX
    OUT: DataFrame
    '''    
    return pd.read_excel(file_xlsx)

### Extracción principal de datos:

In [ ]:

def extract():
    '''Función para la extracción de datos des de los diferentes tipos de documentos

    IN: None
    OUT: DataFrame, 
    '''
    ########
    #Extraer la información de los documentos:
    ########


    #Damos la ruta al directorio de datos "sucios":
    dirty_data = 'dirty_data/*'

    #Generamos una lista que contendra el total de los DF:
    list_of_dfs = []

    #Diferenciamos por tipo de documento para poder dirigirlo a la función adecuada:
    for doc in glob.glob(dirty_data): ###REMARK
        
        #Generamos los df con las funciones y los añadimos a la lista de df
        if doc[-4:] == ".csv": df_csv = list_of_dfs.append(extractANDclean_CSV(doc))
        elif doc[-5:] == ".json": df_json = list_of_dfs.append(extractANDclean_JSON(doc))
        elif doc[-5:] == ".xlsx": df_xlsx = list_of_dfs.append(extractANDclean_XLSX(doc))
            #Presuponemos que el separador para los TXT será "|":
        elif doc[-4:] == ".txt": df_txt = list_of_dfs.append(extractANDclean_TXT(doc))
        
        
        #Levantamos un error si el formato no existe:
        else: 
            raise TypeError(f"Documento con formato no contemplado: {doc}")

        
        ######
        #Desplazar los documentos utilizados:
        ######
        
        print(doc[0:11]+'clean_'+doc[11:])
        

    #Concatenamos todos los df con todos los registros de los diferentes documentos:
    df_sales= pd.concat(list_of_dfs, ignore_index=True) ###REMARK

In [25]:
extract()

dirty_data/clean_dataset_ventas.csv
dirty_data/clean_dataset_ventas.xlsx
dirty_data/clean_dataset_ventas.txt
dirty_data/clean_dataset_ventas.json
